In [10]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

In [2]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.mean()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [4]:
X = np.array(X)
X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
#y = scaler.fit_transform(X)
des=X[:,:13]
body=X[:,13:]

In [25]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 10, shuffle = True, random_state = 111)
kfold.get_n_splits(X)

10

In [6]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [30]:
dict_x = dict()
dict_y = dict()

class train_model:
    def __init__(self, x, y, pd_des, pd_body):
        self.X = x
        self.y = y
        self.pd_des = pd_des
        self.pd_body = pd_body

        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            X_train, X_test = self.X[train_index], self.X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            svm = SVC(kernel = 'poly' , C = 10.0 , gamma = 0.1)#, class_weight = weight
            svm.fit(X_train ,y_train)
            pred_y = svm.predict(X_test)
   
            print(pred_y)

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_des[index] = pred_y[i]

        return self.pd_des
    '''
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_sm_y[index] = y_pred[i]
            
        return self.pd_sm_y
    '''
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            X_train, X_test = self.X[train_index], self.X[test_index]
            y_train, y_test = y[train_index], y[test_index]
                        
            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            svm = SVC(kernel = 'poly' , C = 10.0 , gamma = 0.1, class_weight = weight)#, class_weight = weight
            svm.fit(X_train ,y_train)
            pred_des = svm.predict(X_test)

            pred_des = pred_des[:,1]
            
            print(pred_des)

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_des[index] = pred_des[i]

        return self.pd_des

    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            X_train, X_test = self.X[train_index], self.X[test_index]
            y_train, y_test = y[train_index], y[test_index]
                        
            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            sm_X_train, sm_y_train = sm.fit_resample(X_train, y_train)
            
            svm = SVC(kernel = 'poly' , C = 10.0 , gamma = 0.1, class_weight = weight)#, class_weight = weight
            svm.fit(sm_X_train ,sm_y_train)
            pred_des = svm.predict(X_test)


            pred_des = rf_des.predict_proba(X_test)

            pred_des = pred_des[:,1]
            print(pred_des)

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_des[index] = pred_des[i]


        return self.pd_des

In [31]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        print(confusion.ravel())
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy) #pb
        print("Precision(pb) :",mean_precision )
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb

        print("MCC :", mcc)
        print("G-mean :", g_mean)


        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 

        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [32]:
svm_y, svm_sm_y, svm_cw_y, svm_smcw_y = [], [], [], []
output_svm = train_model(X, y, dict_x, dict_y)
svm_y = output_rf.train_based()
final_result = Evaluation(predict_change(svm_y), y)

TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
[1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1
 1 1 1 0 1 1 1 1 1]
TEST: [  5  39  46  52  68 102 104 122 162 164 166 167 223 228 230 234 249 258
 277 280 281 291 326 331 332 338 342 366 374 375 381 383 389 401 413 421
 423 431 443 444 460 465 471 481 493 498 507 516 526 537 551 572 575 581
 586 604 606 613 632 651 652 653 664 666 673 694 701 706 707 709 718 723
 744 745 754 767 769 780 783 789 806 816 822]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [33]:
final_result.matrix()

[ 15  56  81 676]
공통 
AUC : 0.55213314231492
AUPRC : 0.9571610168560608
Optimized precision : 0.13838146791824

function 사용
Accuracy : 0.8345410628019324
Precision(pb) : 0.9459161584963439
Recall(pb) : 0.6309995596653457
F1 score(pb) : 0.7570127846621257
MCC : 0.09118635489299168
G-mean : 0.4343520378053286

Confusion_matrix 사용 
 [[ 15  56]
 [ 81 676]]
Accuracy : 0.8345410628019324
Precision : 0.9234972677595629
Recall : 0.892998678996037
F1 score : 0.9079919408999328
MCC : 0.16540137357017495
G-mean : 0.4343520378053286


C:\Users\user\anaconda3\envs\gpu\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


(0.55213314231492,
 0.9571610168560608,
 0.13838146791824,
 0.8345410628019324,
 0.9459161584963439,
 0.6309995596653457,
 0.7570127846621257,
 0.09118635489299168,
 0.4343520378053286,
 0.8345410628019324,
 0.9234972677595629,
 0.892998678996037,
 0.9079919408999328,
 0.16540137357017495,
 0.4343520378053286)